In [ ]:
import numpy as np
import networkx as nx
import pandas as pd

def compute_eigenvector_centrality_for_disconnected(graph):
    """
    Compute eigenvector centrality for a potentially disconnected graph.
    Handles strongly connected components separately and merges the results.
    """
    centrality = {}
    for component in nx.strongly_connected_components(graph):
        subgraph = graph.subgraph(component)
        adjacency_matrix = nx.to_numpy_array(subgraph)
        eigenvalues, eigenvectors = np.linalg.eig(adjacency_matrix)
        largest_idx = np.argmax(np.real(eigenvalues))
        eigenvector = np.real(eigenvectors[:, largest_idx])
        centrality.update({node: eigenvector[i] for i, node in enumerate(subgraph.nodes())})
    return centrality

def compute_network_vulnerability_index(graph):
    """
    Compute Network Vulnerability Index (NVI) for a single connected component of the graph.
    """
    degrees = dict(graph.in_degree())
    N = len(degrees)
    D_inv = np.diag([1 / d if d != 0 else 0 for d in degrees.values()])
    c = compute_eigenvector_centrality_for_disconnected(graph)
    c_vector = np.array([c[node] for node in graph.nodes()])
    ones = np.ones(N)
    alpha = 0.5
    v = D_inv @ ones + alpha * c_vector
    NVI = np.mean(v)
    return NVI

def compute_nvi_for_weakly_connected_graph(graph):
    """
    Compute the overall Network Vulnerability Index (NVI) for a weakly connected graph.
    """
    components = list(nx.weakly_connected_components(graph))
    total_nvi = 0
    total_nodes = 0
    for i, component in enumerate(components):
        subgraph = graph.subgraph(component).copy()
        num_nodes = subgraph.number_of_nodes()
        if num_nodes > 1:
            print(f"Processing component {i + 1} with {num_nodes} nodes and {subgraph.number_of_edges()} edges")
            try:
                component_nvi = compute_network_vulnerability_index(subgraph)
                total_nvi += component_nvi * num_nodes
                total_nodes += num_nodes
                print(f"Component {i + 1} NVI: {component_nvi:.4f}")
            except nx.NetworkXError as e:
                print(f"Error processing component {i + 1}: {e}")
        else:
            print(f"Skipping component {i + 1} with only one node.")
    overall_nvi = total_nvi / total_nodes if total_nodes > 0 else 0
    return overall_nvi

In [ ]:
print("\nProcessing R dependency graph...")
r_dependency = pd.read_csv("Rdependency.csv")
r_graph = make_r_graph(r_dependency)
overall_nvi_r = compute_nvi_for_weakly_connected_graph(r_graph)
print(f"\nOverall Network Vulnerability Index (NVI) for R Graph: {overall_nvi_r:.4f}")
print("\nProcessing Python dependency graph...")
python_dependency = pd.read_csv("pythondependency.csv")
python_graph = make_python_graph(python_dependency)
overall_nvi_python = compute_nvi_for_weakly_connected_graph(python_graph)
print(f"\nOverall Network Vulnerability Index (NVI) for Python Graph: {overall_nvi_python:.4f}")
print("\nProcessing Java dependency graph...")
java_dependency = pd.read_csv('Javadependency.csv', delimiter='=>', header=None, names=['from', 'to'], engine='python')
java_graph = make_java_graph(java_dependency)
overall_nvi_java = compute_nvi_for_weakly_connected_graph(java_graph)
print(f"\nOverall Network Vulnerability Index (NVI) for Java Graph: {overall_nvi_java:.4f}")